In [ ]:
from utils.molec_class_files import r14, r32, r50, r125, r134a, r143a, r170, r41, r23, r161, r152a, r152, r134, r143, r116
from utils import atom_type, opt_atom_types
import numpy as np
import unyt as u
import pandas as pd
from fffit.fffit.utils import values_real_to_scaled, values_scaled_to_real, variances_scaled_to_real, generate_lhs
from fffit.fffit.plot import plot_obj_contour
import os

In [ ]:
#Set params for what you want to analyze
save_data = False #Data to save
obj_choice = "ExpValPrior" #Objective to consider
at_number = 11 #atom type to consider
seed = 1 #Seed to use
molec_names = ["R14", "R32", "R50", "R170", "R125", "R134a", "R143a"] #Training data to consider

#Create visualization object
visual = opt_atom_types.Vis_Results(molec_names, at_number, seed, obj_choice)
#Set parameter set of interest (in this case get the best parameter set)
x_label = "best_set"
all_molec_dir = visual.use_dir_name
path_best_sets = os.path.join(all_molec_dir, "best_per_run.csv")
print(path_best_sets)
assert os.path.exists(path_best_sets), "best_per_run.csv not found in directory"
all_df = pd.read_csv(path_best_sets, header = 0)
first_param_name = visual.at_class.at_names[0] + "_min"
last_param_name = visual.at_class.at_names[-1] + "_min"
all_sets = all_df.loc[:, first_param_name:last_param_name].values
unique_best_sets = visual.get_unique_sets(all_sets, 0.05, save_data, save_label=x_label)
print(unique_best_sets)

In [ ]:
#Load in molecgp_data R32 vap gp from pkl file
molec_names = ["R14", "R32", "R50", "R170", "R125", "R134a", "R143a"]
molecgp_data = opt_atom_types.get_gp_data_from_pkl(molec_names)
model = molecgp_data["R14"]['sim_Hvap']
hyperparameters = {
        'Mean Fxn A': model.mean_function.A.numpy(),
        'Mean Fxn B': model.mean_function.b.numpy(),
        'kernel_variance': model.kernel.variance.numpy(),
        'kernel_lengthscale': model.kernel.lengthscales.numpy(),
        'likelihood_variance': model.likelihood.variance.numpy()
    }
print(hyperparameters)

In [ ]:
#Imports
from utils.molec_class_files import r14, r32, r50, r125, r134a, r143a, r170
from utils import atom_type, opt_atom_types
import numpy as np
import unyt as u
import pandas as pd
import os
import copy
import scipy 
import signac
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

#Set params for what you want to analyze
save_data = False #Data to save
obj_choice = "ExpValPrior" #Objective to consider
at_number = 11 #atom type to consider
seed = 1 #Seed to use
molec_names = ["R14", "R32", "R50", "R170", "R125", "R134a", "R143a"] #Training data to consider

#Get best_run data saved in one csv from all jobs
project = signac.get_project("opt_at_params")
#Create visualization object
visual = opt_atom_types.Vis_Results(molec_names, at_number, seed, obj_choice)

def get_pref_theta(molec_names, at_number_list, obj_choices_list, old_list, which_list,  param_set):
    mapds_dfs = []

    for at_number, obj_choice, old, which, param in zip(at_number_list,obj_choices_list, old_list, which_list, param_set):
        visual = opt_atom_types.Vis_Results(molec_names, at_number, seed, obj_choice)
        all_molec_dir = visual.use_dir_name
        if old:
            all_molec_dir = str(all_molec_dir).replace("Results", "Results-OldGP")
        try:
            path_best_sets = os.path.join(all_molec_dir, "unique_best_set.csv")
            all_df = pd.read_csv(path_best_sets, header = 0)
            first_param_name = visual.at_class.at_names[0]
            last_param_name = visual.at_class.at_names[-1]
            best_set = all_df.loc[param, first_param_name:last_param_name].values
        except:
            path_best_sets = os.path.join(all_molec_dir, "best_per_run.csv")
            all_df = pd.read_csv(path_best_sets, header = 0)
            first_param_name = visual.at_class.at_names[0] + "_min"
            last_param_name = visual.at_class.at_names[-1] + "_min"
            best_set = all_df.loc[param, first_param_name:last_param_name].values
        best_real = visual.values_pref_to_real(copy.copy(best_set))
        # mapds = visual.calc_MAPD_mimic_ms(molec_names, best_set)
        dfs = visual.calc_MAPD_best(molec_names, best_real)
        mapds_dfs.append(dfs[which])

    return mapds_dfs

at_schemes = [11, 11, 11]
old_list = [False, False, False]
objs = ["ExpValPrior", "ExpValPrior", "ExpValPrior"]
which = ["Literature", "Opt_All", "Opt_All"]
param_set = [0,0,1]
mapd_dfs = get_pref_theta(molec_names, at_schemes, objs, old_list, which, param_set)
# mapd_dfs = [pd.read_csv(file, header =0 ) for file in files]
df_labels = ["Literature", "5% eps_C1 Wt", "20% eps_C1 Wt"]
df_colors = ["blue", "orange", "green"]
title = "AT 11 E[SSE] + GAFF Prior Results"

visual.plot_avg_MAPD(mapd_dfs, df_labels, df_colors, title)
visual.plot_MAPD(mapd_dfs, df_labels, df_colors, title)

In [ ]:
import scipy 
import os
repeats = 1
seed = 1
#Get obj from a set of parameters
at_class = 11
save_data = False
obj_choice = "ExpVal"
obj_choice_p = "ExpValPrior"
molec_names = ["R14", "R32", "R50", "R125", "R134a", "R143a", "R170"]
at_optimizer =  opt_atom_types.Opt_ATs(molec_names, at_class, repeats, seed, obj_choice_p)
# visual = opt_atom_types.Vis_Results(molec_data_dict, all_gp_dict, at_class, obj_choice)
# visual.check_GPs()
setup = opt_atom_types.Problem_Setup(molec_names, at_class, obj_choice)
all_molec_dir = setup.use_dir_name
all_df = pd.read_csv(all_molec_dir /"best_per_run.csv", header = 0)
sorted_df = all_df.sort_values(by='Min Obj', ascending = True).reset_index(drop = True)
first_param_name = setup.at_class.at_names[0] + "_min"
last_param_name = setup.at_class.at_names[-1] + "_min"
best_pref = all_df.loc[0, first_param_name:last_param_name].values
best_real= at_optimizer.values_pref_to_real(best_pref)
# best_real = generate_lhs(1, at_optimizer.at_class.at_bounds_nm_kjmol, at_optimizer.seed, labels = None)
obj = at_optimizer.calc_obj(best_real)
print(obj[0])

# #Get Jac and Hess info from a set of parameters
# dir_name = at_optimizer.make_results_dir(list(at_optimizer.molec_data_dict.keys()))
# save_path = os.path.join(dir_name, "best_set" + "_jac_approx.npy")
# hess = np.load(save_path)
# print(hess)
# jac = at_optimizer.approx_jac(best_real)
# hess = at_optimizer.approx_hess(best_real)
# print(scipy.linalg.norm(jac, ord = np.inf))
# print(jac)
# print(hess)
# eigval, eigvec = scipy.linalg.eig(hess)
# print(eigval)

In [ ]:

#Imports
from utils.molec_class_files import r14, r32, r50, r125, r134a, r143a, r170
from utils import atom_type, opt_atom_types
import numpy as np
import unyt as u
import pandas as pd
import os
import copy
import scipy 
import signac
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

#Set params for what you want to analyze
save_data = False #Data to save
obj_choice = "ExpValPrior" #Objective to consider
at_number = 11 #atom type to consider
seed = 1 #Seed to use
molec_names = ["R14", "R32", "R50", "R170", "R125", "R134a", "R143a"] #Training data to consider

#Get best_run data saved in one csv from all jobs
project = signac.get_project("opt_at_params")
#Create visualization object
visual = opt_atom_types.Vis_Results(molec_names, at_number, seed, obj_choice)

#Set parameter set of interest (in this case get the best parameter set)
all_molec_dir = visual.use_dir_name
path_best_sets = os.path.join(all_molec_dir, "best_per_run.csv")
assert os.path.exists(path_best_sets), "best_per_run.csv not found in directory"
all_df = pd.read_csv(path_best_sets, header = 0)
first_param_name = visual.at_class.at_names[0] + "_min"
last_param_name = visual.at_class.at_names[-1] + "_min"
best_set = all_df.loc[0, first_param_name:last_param_name].values
# path_best_sets = os.path.join(all_molec_dir, "opt_params_rcc_best_set_1.csv")
# assert os.path.exists(path_best_sets), "opt_params_rcc_best_set_1.csv not found in directory"
# all_df = pd.read_csv(path_best_sets, header = 0)
# best_set = all_df.loc[:, visual.at_class.at_names].values
best_real = visual.values_pref_to_real(copy.copy(best_set))
x_label = "best_set"

# pdf = PdfPages('Results/at_schemes_' + at_str + '.pdf')
visual.calc_MAPD_best2(molec_names)
visual.plot_MSE_ind(molec_names, label = x_label)
# visual.plot_at_MSE(molec_names, at_schemes)
#Close figures 
# plt.close()
# pdf.close()   